In [2]:
import os
import cv2 
import time
import json
import math
import copy
import torch
import numpy as np
from vidstab import VidStab
import matplotlib.pyplot as plt

from lightglue import viz2d
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd, load_image_from_path
import CSRansac

from vidstab import VidStab

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
#matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).eval().to(device)
matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).eval().to(device)
#matcher.compile(mode='reduce-overhead')

In [5]:
print(torch.__version__)
print(device)

2.1.2
cuda


In [6]:
aircraft_datasets = "datasets"

lables = os.path.join(aircraft_datasets + "/label")
video_dir = os.path.join(aircraft_datasets, "video")
target_image_dir = os.path.join(aircraft_datasets, "target_image")
# 비디오 안정화 객체 생성
stabilizer = VidStab()

In [12]:
import cv2

disappear_error = 0
misannotate_error = 0
pixel_error = 0

missing_inlier = 0
failed_inliers = 0

x = 637 // 2
y = 367 // 2

# 저장할 동영상 파일명 및 코덱 설정
output_video_path = 'lightglue_unstable_result.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_fps = 30.0  # 저장할 동영상의 프레임 속도

cap = cv2.VideoCapture('demo_video_resized.mp4')
image0 = load_image("img0.png", grayscale=True)

# 저장할 동영상의 너비와 높이 설정
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter 객체 생성
out = cv2.VideoWriter(output_video_path, fourcc, output_video_fps, (frame_width, frame_height))

count = 0

# 각 프레임을 VideoWriter에 쓰기
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임에 작업 수행
    if count == 0:
        count += 1
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('frame', frame)
        continue
    feats0 = extractor.extract(image0.to(device))
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, mask = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    if mask < 0.3:
        out.write(frame)
        cv2.imshow('frame', frame)
        failed_inliers += 1
        continue
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    #image0 = image1
    
    cv2.circle(frame, (int(projected_pts[0]), int(projected_pts[1])), 5, (0, 0, 255), -1)

    # 동영상 파일에 프레임 추가
    out.write(frame)

    # 화면에 표시
    cv2.imshow('frame', frame)
    
    float_x = x / 640
    float_y = y / 480
    _x = int(projected_pts[0]) / 640
    _y = int(projected_pts[1]) / 480
    
    if _x < 0 or _x > 1 or _y < 0 or _y > 1:
        disappear_error += 1
        
    distance = np.sqrt((_x - float_x) ** 2 + (_y - float_y) ** 2)
    if distance > 0.1:
        misannotate_error += 1
        
    if distance > pixel_error:
        pixel_error = distance
    
    key = cv2.waitKey(5)
    if key == 27:
        break

# VideoWriter 객체 해제
out.release()

# VideoCapture 객체 해제
cap.release()

# 모든 창 닫기
cv2.destroyAllWindows()


In [13]:
print("disappear_error:", disappear_error)
print("misannotate_error:", misannotate_error)
print("pixel_error:", pixel_error)  
print("missing_inlier:", missing_inlier)
print("failed_inliers:", failed_inliers)

disappear_error: 0
misannotate_error: 0
pixel_error: 0.06406250000000002
missing_inlier: 0
failed_inliers: 36


## 동영상을 이미지로

In [25]:
# 폴더 경로 설정
output_dir = 'stable_005_video'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 동영상 파일 로드
video = cv2.VideoCapture("stable_movie_005.mp4")

# 프레임 카운터 초기화
frame_count = 0

while True:
    # 동영상에서 프레임을 읽음
    ret, frame = video.read()
    if not ret:
        break  # 동영상 끝에 도달하면 중단
    
    # 프레임을 이미지 파일로 저장
    frame_filename = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    frame = cv2.resize(frame, (640, 480))
    cv2.imwrite(frame_filename, frame)
    
    frame_count += 1

# 자원 해제
video.release()

In [23]:
from vidstab import VidStab

# Using defaults
stabilizer = VidStab()
stabilizer.stabilize(input_path='movie_006.mp4', output_path='stable_movie_006.mp4')

## 이미지를 동영상으로"

In [20]:
# 폴더 경로 설정
folder = 'dump_demo_sequence/'

# 동영상 저장 경로 설정
output_video_path = 'result_origin.mp4'

# 동영상 속성 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정 (XVID를 사용하면 AVI 형식으로 저장)
fps = 30.0  # 초당 프레임 수
frame_width = 320  # 프레임 너비
frame_height = 480  # 프레임 높이

out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

i = 0
for name in os.listdir(folder):
    img = cv2.imread(os.path.join(folder, name))
    
    cv2.imshow('frame', img)
    
    # 프레임을 동영상에 추가
    out.write(img)
    
    
    # 종료 키 입력 확인
    key = cv2.waitKey(5)
    if key == 27:
        break

# 동영상 저장 종료
out.release()
cv2.destroyAllWindows()


In [10]:
import cv2

x = 319
y = 184

# 저장할 동영상 파일명 및 코덱 설정
output_video_path = 'stabled_005.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_fps = 30.0  # 저장할 동영상의 프레임 속도

cap = cv2.VideoCapture('stable_movie_005.mp4')
image0 = load_image("img1.png", grayscale=True)

# 저장할 동영상의 너비와 높이 설정
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter 객체 생성
out = cv2.VideoWriter(output_video_path, fourcc, output_video_fps, (frame_width, frame_height))

count = 0

# 각 프레임을 VideoWriter에 쓰기
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임에 작업 수행
    if count == 0:
        count += 1
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('frame', frame)
        continue
    feats0 = extractor.extract(image0.to(device))
    image1 = stabilizer.stabilize_frame(input_frame = frame)
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, mask = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    if mask < 0.3:
        out.write(frame)
        cv2.imshow('frame', frame)
        continue
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    #image0 = image1
    
    cv2.circle(frame, (int(projected_pts[0]), int(projected_pts[1])), 5, (0, 0, 255), -1)

    # 동영상 파일에 프레임 추가
    out.write(frame)

    # 화면에 표시
    cv2.imshow('frame', frame)
    
    # 종료 키 입력 확인
    key = cv2.waitKey(5)
    if key == 27:
        break

# VideoWriter 객체 해제
out.release()

# VideoCapture 객체 해제
cap.release()

# 모든 창 닫기
cv2.destroyAllWindows()


In [12]:
import cv2

x = 637 // 2
y = 367 // 2

# 저장할 동영상 파일명 및 코덱 설정
output_video_path = 'lightglue_unstable_result.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_fps = 30.0  # 저장할 동영상의 프레임 속도

cap = cv2.VideoCapture('demo_video.mp4')
image0 = load_image("img0.png", grayscale=True)

# 저장할 동영상의 너비와 높이 설정
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter 객체 생성
out = cv2.VideoWriter(output_video_path, fourcc, output_video_fps, (frame_width, frame_height))

count = 0

# 각 프레임을 VideoWriter에 쓰기
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임에 작업 수행
    if count == 0:
        count += 1
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('frame', frame)
        continue
    feats0 = extractor.extract(image0.to(device))
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, mask = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    if mask < 0.3:
        out.write(frame)
        cv2.imshow('frame', frame)
        continue
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    #image0 = image1
    
    cv2.circle(frame, (int(projected_pts[0]), int(projected_pts[1])), 5, (0, 0, 255), -1)

    # 동영상 파일에 프레임 추가
    out.write(frame)

    # 화면에 표시
    cv2.imshow('frame', frame)
    
    # 종료 키 입력 확인
    key = cv2.waitKey(5)
    if key == 27:
        break

# VideoWriter 객체 해제
out.release()

# VideoCapture 객체 해제
cap.release()

# 모든 창 닫기
cv2.destroyAllWindows()


IndexError: list assignment index out of range

: 

## 속도 측정(인접 프레임)

In [9]:
import cv2
import time

x = 637 // 2
y = 367 // 2

image0 = load_image("img0.png", grayscale=True)
count = 0

# 프레임 수 초기화
frame_count = 0

failed_inliers = 0

# 프레임 별 처리 시간 리스트 초기화
frame_processing_times = []

cap = cv2.VideoCapture('demo_video_resized.mp4')

# 각 프레임을 VideoWriter에 쓰기
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time = time.time()

    # 프레임에 작업 수행
    if count == 0:
        count += 1
        
        continue
    feats0 = extractor.extract(image0.to(device))
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, mask = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    if mask < 0.3:
        failed_inliers += 1
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    image0 = image1
    
    cv2.circle(frame, (int(projected_pts[0]), int(projected_pts[1])), 5, (0, 0, 255), -1)
    cv2.imshow('frame', frame)
    
    # 현재 시간 측정
    current_time = time.time()
    
    # 프레임 처리 시간 계산
    frame_processing_time = current_time - start_time
    frame_processing_times.append(frame_processing_time)
    
    # 이전 프레임 처리 시간 업데이트
    prev_frame_time = current_time

    # FPS 계산
    fps = 1.0 / frame_processing_time
    
    # 프레임 수 증가
    frame_count += 1
    
    key = cv2.waitKey(5)
    if key == 27:
        break

# 모든 창 닫기
cv2.destroyAllWindows()

#전체 처리 시간 계산
total_processing_time = sum(frame_processing_times)

# 전체 프레임 수와 전체 처리 시간을 사용하여 평균 FPS 계산
average_fps = frame_count / total_processing_time

print(f"Total Frames Processed: {frame_count}")
print(f"Average FPS: {average_fps:.2f}")


Total Frames Processed: 367
Average FPS: 12.47


## 속도 측정 (타깃 프레임)

In [24]:
import cv2
import time

x = 637 // 2
y = 367 // 2

image0 = load_image("img0.png", grayscale=True)
count = 0

# 프레임 수 초기화
frame_count = 0

# 프레임 별 처리 시간 리스트 초기화
frame_processing_times = []

cap = cv2.VideoCapture('demo_video_resized.mp4')
image0 = load_image("img0.png", grayscale=True)

# 각 프레임을 VideoWriter에 쓰기
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time = time.time()
    
    feats0 = extractor.extract(image0.to(device))
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, mask = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    #image0 = image1
    
    # 현재 시간 측정
    current_time = time.time()
    
    # 프레임 처리 시간 계산
    frame_processing_time = current_time - start_time
    frame_processing_times.append(frame_processing_time)
    
    # 이전 프레임 처리 시간 업데이트
    prev_frame_time = current_time

    # FPS 계산
    fps = 1.0 / frame_processing_time
    
    # 프레임 수 증가
    frame_count += 1
    
    key = cv2.waitKey(5)
    if key == 27:
        break

# 모든 창 닫기
cv2.destroyAllWindows()

#전체 처리 시간 계산
total_processing_time = sum(frame_processing_times)

# 전체 프레임 수와 전체 처리 시간을 사용하여 평균 FPS 계산
average_fps = frame_count / total_processing_time

print(f"Total Frames Processed: {frame_count}")
print(f"Average FPS: {average_fps:.2f}")


Total Frames Processed: 368
Average FPS: 7.35
